In [2]:
# Libraries in main file
import datetime
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidSessionIdException

from Config import params_config, db_config
from ScrapingTools.KeibaLabScraper import RaceInfoScraper

import warnings
warnings.filterwarnings('ignore')

# Libraries in KeibaLabScraper.py
import os
import sys
import re
import time
import copy
import urllib3
import pymysql
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from Utils.bulk_insert import BulkInsert

In [3]:
parameters = params_config.parameters
db_params = db_config.db_params
con = pymysql.connect(**db_params)

In [4]:
def initialize_chrome_driver(parameters):
    chrome_options = Options()
    chrome_options.add_argument('--dns-prefetch-disable')
    driver = Chrome(executable_path=parameters['DRIVER_DIR'], chrome_options=chrome_options)
    driver.set_page_load_timeout(parameters['PAGE_LOAD_TIMEOUT'])
    driver.maximize_window()
    return driver

In [20]:
driver = initialize_chrome_driver(parameters)

## HorseInfoScraper
### scraping_horse_info_not_acquired

In [18]:
# Functions
def _back_chrome_window_for_number_of(times):
    for _ in range(times):
        driver.back()
    return None

def _load_target_url_page(target_url):
    try:
        driver.get(target_url)
        print('We could load the URL:', driver.current_url)
    except (TimeoutException, urllib3.exceptions.MaxRetryError) as e:
        print(e)
        driver.refresh()

def _make_web_driver_click_by(xpath, verbose=True):
    for i in range(parameters['RETRIES_WHEN_WEB_CLICK']):
        try:
            WebDriverWait(driver, parameters['PAGE_LOAD_TIMEOUT']).until(
                EC.element_to_be_clickable((By.XPATH, xpath)))
            driver.find_element_by_xpath(xpath).click()
        except TimeoutException:
            print('Timeout when web_driver_click, so retrying... ({TIME}/{MAX})'.
                  format(TIME=i+1, MAX=parameters['RETRIES_WHEN_WEB_CLICK']))
            continue
        else:
            if verbose:
                print('We could load the XPATH and now locate in:', driver.current_url)
            return None
    raise NoSuchElementException\

def _fetchall_query(query):
    try:
        cursor = con.cursor()
        cursor.execute(query)
        fetch_result = cursor.fetchall()
        cursor.close()
        return fetch_result
    except Exception as e:
        print(e)

In [10]:
def get_not_acquired_href_to_horse(db_params, con):
    select_query = '''
        SELECT DISTINCT href_to_the_horse
        FROM keibalab_race_prior_info_list;
    '''
    fetch_result = _fetchall_query(select_query)
    return list(fetch_result)

In [14]:
fecth_result = get_not_acquired_href_to_horse(db_params, con)
href_to_horse_list = [item[0] for item in fecth_result]

In [15]:
i = 0
href_to_horse = href_to_horse_list[0]

In [16]:
href_to_horse

'https://www.keibalab.jp/db/horse/2013106098/'

In [21]:
_load_target_url_page(target_url=href_to_horse)

We could load the URL: https://www.keibalab.jp/db/horse/2013106098/
